## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/mshar/Class/Week 6/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,77.25,80,55,4.00,broken clouds
1,1,Chilakalurupet,IN,16.0833,80.1667,77.63,92,41,4.38,scattered clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,46.51,93,100,11.50,light rain
3,3,Ancud,CL,-41.8697,-73.8203,53.22,88,95,8.66,light rain
4,4,Aksu,CN,41.1231,80.2644,46.26,53,75,3.53,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,77.25,80,55,4.00,broken clouds
1,1,Chilakalurupet,IN,16.0833,80.1667,77.63,92,41,4.38,scattered clouds
6,6,Puerto Ayora,EC,-0.7393,-90.3518,80.56,78,61,8.95,light rain
11,11,Tessalit,ML,20.1986,1.0114,85.95,11,91,14.25,overcast clouds
17,17,Faanui,PF,-16.4833,-151.7500,82.18,72,47,12.80,scattered clouds
21,21,Bowen,AU,-20.0167,148.2333,79.36,74,64,10.89,broken clouds
23,23,Hithadhoo,MV,-0.6000,73.0833,82.56,69,1,11.18,clear sky
24,24,Saint-Philippe,RE,-21.3585,55.7679,76.59,83,16,7.20,few clouds
25,25,Bambous Virieux,MU,-20.3428,57.7575,79.21,89,40,4.61,scattered clouds
26,26,Port Hedland,AU,-20.3167,118.5667,82.45,74,9,6.91,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                223
City                   223
Country                222
Lat                    223
Lng                    223
Max Temp               223
Humidity               223
Cloudiness             223
Wind Speed             223
Current Description    223
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy() 
clean_df.dropna(inplace = True) 
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,77.25,broken clouds,19.7297,-155.0900,
1,Chilakalurupet,IN,77.63,scattered clouds,16.0833,80.1667,
6,Puerto Ayora,EC,80.56,light rain,-0.7393,-90.3518,
11,Tessalit,ML,85.95,overcast clouds,20.1986,1.0114,
17,Faanui,PF,82.18,scattered clouds,-16.4833,-151.7500,
21,Bowen,AU,79.36,broken clouds,-20.0167,148.2333,
23,Hithadhoo,MV,82.56,clear sky,-0.6000,73.0833,
24,Saint-Philippe,RE,76.59,few clouds,-21.3585,55.7679,
25,Bambous Virieux,MU,79.21,scattered clouds,-20.3428,57.7575,
26,Port Hedland,AU,82.45,clear sky,-20.3167,118.5667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")       
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
clean_hotel_df.dropna(inplace = True)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,77.25,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
1,Chilakalurupet,IN,77.63,scattered clouds,16.0833,80.1667,Sri vishnu priya lodge
6,Puerto Ayora,EC,80.56,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
17,Faanui,PF,82.18,scattered clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
21,Bowen,AU,79.36,broken clouds,-20.0167,148.2333,Castle Motor Lodge Motel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))